# Ružička: Authorship Verification in Python

In this notebook, we offer a quick tutorial as to how you could use the code in this repository. While the package is very much geared towards our own work in authorship verification, you might some of the more general functions useful. All feedback and comments are welcome. This code assumes Python 2.7+ (Python 3 has not been tested). You do not need to install the library to run the code below, but please note that there are a number of well-known third-party Python libraries, including:
+ numpy
+ scipy
+ scikit-learn
+ matplotlib
+ seaborn
+ numba

and preferably (for GPU acceleration and/or JIT-compilation):
+ theano
+ numbapro

We recommend installing Continuum's excellent [Anaconda Python framework](https://www.continuum.io/downloads), which comes bundled with most of these dependencies.


In [1]:
import logging

logging.basicConfig(level="INFO")

## Walk through

By default, we assume that your data sets are stored in a directory the format on the PAN 2014 track on authorship attribution: a directory should minimally include one folder per verification problem (an `unknown.txt` and at least one `known01.txt`) and a `truth.txt`. E.g. for the corpus of Dutch essays (`../data/2014/du_essays/train`), `truth.txt` contains has a tab-separated line with the ground truth for each problem:

```
DE001 Y
DE002 Y
DE003 N
DE004 N
DE005 N
DE006 N
DE007 N
DE008 Y
...
```

To inspect the problems:

In [2]:
! ls ../data/2014/du_essays/train

DE001         DE021         DE041         DE061         DE081
DE002         DE022         DE042         DE062         DE082
DE003         DE023         DE043         DE063         DE083
DE004         DE024         DE044         DE064         DE084
DE005         DE025         DE045         DE065         DE085
DE006         DE026         DE046         DE066         DE086
DE007         DE027         DE047         DE067         DE087
DE008         DE028         DE048         DE068         DE088
DE009         DE029         DE049         DE069         DE089
DE010         DE030         DE050         DE070         DE090
DE011         DE031         DE051         DE071         DE091
DE012         DE032         DE052         DE072         DE092
DE013         DE033         DE053         DE073         DE093
DE014         DE034         DE054         DE074         DE094
DE015         DE035         DE055         DE075         DE095
DE016         DE036         DE056         DE076         DE096
DE017   

Let us now load the set of development problems for the Dutch essays:

In [3]:
from ruzicka.utilities import *

D = "../data/2014/du_essays/"
dev_train_data, dev_test_data = load_pan_dataset(D + "train")

This functions loads all documents and splits the development data into a development part (the known documents) and a testing part (the unknown documents). We can unpack these as follows:

In [4]:
dev_train_labels, dev_train_documents = zip(*dev_train_data)
dev_test_labels, dev_test_documents = zip(*dev_test_data)

Let us have a look at the actual test texts:

In [5]:
for doc in dev_test_documents[:10]:
    print("+ ", doc[:70])

+  ﻿Dankzij het internet zijn we een grote bron aan informatie rijker . A
+  ﻿Het is dus begrijpelijk dat de commerciële zenders meer reclame moete
+  ﻿" Hey , vuile nicht ! Hangt er nog stront aan je lul ? " . Dergelijke
+  ﻿Gelijkheid tussen man en vrouw is iets dat ons al eeuwen in de ban ho
+  ﻿Gisteren was er opnieuw een protest tegen homofilie in de grootstad P
+  ﻿Voetbal is vandaag de dag zonder twijfel de populairste sport in Belg
+  ﻿Door de ongekende groei van nieuwsbronnen en de opkomst van het inter
+  ﻿Woordenboekgebruik uit interesse De categorie woordenboekgebruikers d
+  ﻿Ze bouwden een tegencultuur op die alles verwierp waar hun ouders alt
+  ﻿Als we hier in België op straat rondlopen , merken we dat er zeer vee


For each of these documents we need to decide whether or not they were in fact written by the target authors proposed:

In [6]:
for doc in dev_test_labels[:10]:
    print("+ ", doc[:70])

+  DE001
+  DE002
+  DE003
+  DE004
+  DE005
+  DE006
+  DE007
+  DE008
+  DE009
+  DE010


The first and crucial step is to vectorize the documents using a vector space model. Below, we use generic example, using the 10,000 most common word unigrams and a plain *tf* model:

In [7]:
from ruzicka.vectorization import Vectorizer

vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)

dev_train_X = vectorizer.fit_transform(dev_train_documents)
dev_test_X = vectorizer.transform(dev_test_documents)

In [8]:
dev_test_X.__class__

numpy.ndarray

Note that we use `sklearn` conventions here: we fit the vectorizer only on the vocabulary of the known documents and apply it it later to the unknown documents (since in real life too, we will not necessarily know the known documents in advance). This gives us two compatible corpus matrices:

In [9]:
print(dev_train_X.shape)
print(dev_test_X.shape)

(172, 9347)
(96, 9347)


We now encode the author labels in the development problem sets as integers, using sklearn's convenient `LabelEncoder`:

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(dev_train_labels + dev_test_labels)
dev_train_y = np.array(label_encoder.transform(dev_train_labels))
dev_test_y = np.array(label_encoder.transform(dev_test_labels))
print(dev_test_y)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


We now construct and fit an 'O2' verifier: this extrinsic verification technique is based on the General Imposters framework. We apply it with the minmax metric and a profile base, meaning that the known documents for each author will be represented as a mean centroid:

In [11]:
from ruzicka.Order2Verifier import Order2Verifier

dev_verifier = Order2Verifier(
    metric="minmax", base="profile", rank=True, nb_bootstrap_iter=100, rnd_prop=0.35
)
dev_verifier.fit(dev_train_X, dev_train_y)

We can now obtain the probability which this O1 verifier would assign to each combination of an unknown document and the target author suggested in the problem:

In [12]:
dev_test_scores = dev_verifier.predict_proba(
    test_X=dev_test_X, test_y=dev_test_y, nb_imposters=30
)

07/21/2023 02:18:57 [ruzicka:INFO] # test documents processed: 10 out of 96
07/21/2023 02:18:57 [ruzicka:INFO] # test documents processed: 20 out of 96
07/21/2023 02:18:57 [ruzicka:INFO] # test documents processed: 30 out of 96
07/21/2023 02:18:58 [ruzicka:INFO] # test documents processed: 40 out of 96
07/21/2023 02:18:58 [ruzicka:INFO] # test documents processed: 50 out of 96
07/21/2023 02:18:58 [ruzicka:INFO] # test documents processed: 60 out of 96
07/21/2023 02:18:58 [ruzicka:INFO] # test documents processed: 70 out of 96
07/21/2023 02:18:58 [ruzicka:INFO] # test documents processed: 80 out of 96
07/21/2023 02:18:59 [ruzicka:INFO] # test documents processed: 90 out of 96


This gives us as an array of probability scores for each problem, corresponding to the number of iterations in which the target's author's profile was closer to the anonymous document than to one of the imposters:

In [13]:
print(dev_test_scores)

[0.75358965 0.70353211 0.08879798 0.0732806  0.2974103  0.161985
 0.11846959 0.995      0.995      0.78442063 0.60322655 0.3905101
 0.89333333 0.9425     0.09385077 0.72146825 0.70299101 0.05577025
 0.19989368 0.53658474 0.08663092 0.06130266 0.59212103 0.47587956
 0.12250346 0.81383333 0.17062584 0.63791209 0.16567762 0.12824681
 0.07325937 0.07179835 0.459573   0.08222499 0.05867227 0.47490065
 0.07409308 0.18723689 0.25447591 0.05248238 0.27360082 0.83028943
 0.1190274  0.7882619  0.98666667 0.04531036 0.50420403 0.15220609
 0.06319807 0.40273145 0.11989127 0.52161849 0.22113008 0.5668017
 0.75158333 0.10384333 0.15120377 0.77019048 0.11440043 0.85074242
 0.96166667 0.62213362 0.92783333 0.44565761 0.56824733 0.92429167
 0.73537042 0.28641346 0.67630159 0.8807619  0.393677   0.12197395
 0.17036251 0.51851587 0.25235617 0.32814221 0.43530778 0.06286593
 0.955      0.06893487 0.87310119 0.35665054 0.04649806 0.05080119
 0.995      0.04929732 1.         0.66844048 0.10608256 0.03889033

Let us now load the ground truth to check how well we did:

In [14]:
dev_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "train", "truth.txt")), labels=dev_test_labels
)
print(dev_gt_scores)

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]


There is one final step needed: the PAN evaluation measures allow systems to leave a number of difficult problems unanswered, by setting the probability exactly at 0.5. To account for this strict threshold, we fit a score shifter, which will attempt to rectify mid-range score to 0.5. We can tune these parameters as follows:

In [15]:
from ruzicka.score_shifting import ScoreShifter

shifter = ScoreShifter()
shifter.fit(predicted_scores=dev_test_scores, ground_truth_scores=dev_gt_scores)
dev_test_scores = shifter.transform(dev_test_scores)

07/21/2023 02:19:02 [ruzicka:INFO] p1 for optimal combo: 0.23
07/21/2023 02:19:02 [ruzicka:INFO] p2 for optimal combo: 0.4
07/21/2023 02:19:02 [ruzicka:INFO] AUC for optimal combo: 0.9505208333333334
07/21/2023 02:19:02 [ruzicka:INFO] c@1 for optimal combo: 0.9456380208333333


As you can see, this shifter optimizes 2 parameters using a grid search: all values in between *p1* and *p2* will be rectified to 0.5:

In [16]:
print(dev_test_scores)

[0.8461713398450343, 0.8149214992862273, 0.01194323523989824, 0.008229820571725902, 0.5, 0.029457381073969963, 0.019043850825898014, 0.996878608022189, 0.996878608022189, 0.8654184598519357, 0.7523029079980297, 0.5, 0.9334103044733651, 0.9641039922551733, 0.013152401549281253, 0.8261186484741627, 0.8145837061729677, 0.004039478054827024, 0.03852918146775924, 0.7106998642531701, 0.011424643348972715, 0.00536342052248279, 0.7453699682800238, 0.6728029333056706, 0.020009183167422295, 0.8837801720261698, 0.03152519150877273, 0.7739563391892905, 0.030341048711086928, 0.02138360571665348, 0.008224739936851662, 0.007875108176681115, 0.662623101042348, 0.010370273948991645, 0.004733950514891202, 0.6721918215982434, 0.008424253777484122, 0.03550032679274277, 0.5, 0.0032526674779033244, 0.5, 0.8940533593918357, 0.01917733881420214, 0.8678164816253651, 0.9916762880591706, 0.001536355298303081, 0.6904852860900496, 0.027117222784225264, 0.005817004155629777, 0.6271381462195903, 0.01938406743782056,

We can later apply this optimized score shifter to the test problems. Now the main question: how well would our O2 verifier perform on the development problems, given the optimal *p1* and *p2* found? We answer this question using the three evaluation measures used in the PAN competition.

In [17]:
from ruzicka.evaluation import pan_metrics

dev_acc_score, dev_auc_score, dev_c_at_1_score = pan_metrics(
    prediction_scores=dev_test_scores, ground_truth_scores=dev_gt_scores
)
print("Accuracy: ", dev_acc_score)
print("AUC: ", dev_auc_score)
print("c@1: ", dev_c_at_1_score)
print("AUC x c@1: ", dev_auc_score * dev_c_at_1_score)

Accuracy:  0.90625
AUC:  0.9505208333333334
c@1:  0.9456380208333333
AUC x c@1:  0.898848639594184


Our score shifting approach clearly pays off, since we are able to leave difficult problems unswered, yielding to a higher c@1 than pure accuracy. We can now proceed to the test problems. The following code block runs entire parallel to the approach above: only the score shifter isn't retrained again:

In [18]:
train_data, test_data = load_pan_dataset(D + "test")
train_labels, train_documents = zip(*train_data)
test_labels, test_documents = zip(*test_data)

# vectorize:
vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)
train_X = vectorizer.fit_transform(train_documents)
test_X = vectorizer.transform(test_documents)

# encode author labels:
label_encoder = LabelEncoder()
label_encoder.fit(train_labels + test_labels)
train_y = np.array(label_encoder.transform(train_labels), dtype="int")
test_y = np.array(label_encoder.transform(test_labels), dtype="int")

# fit and predict a verifier on the test data:
test_verifier = Order2Verifier(
    metric="minmax", base="profile", rank=True, nb_bootstrap_iter=100, rnd_prop=0.35
)
test_verifier.fit(train_X, train_y)
test_scores = test_verifier.predict_proba(
    test_X=test_X, test_y=np.array(test_y), nb_imposters=30
)

# load the ground truth:
test_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "test", "truth.txt")), labels=test_labels
)

# apply the optimzed score shifter:
test_scores = shifter.transform(test_scores)

test_acc_score, test_auc_score, test_c_at_1_score = pan_metrics(
    prediction_scores=test_scores, ground_truth_scores=test_gt_scores
)

print("Accuracy: ", test_acc_score)
print("AUC: ", test_auc_score)
print("c@1: ", test_c_at_1_score)
print("AUC x c@1: ", test_auc_score * test_c_at_1_score)

07/21/2023 02:19:03 [ruzicka:INFO] # test documents processed: 10 out of 96
07/21/2023 02:19:03 [ruzicka:INFO] # test documents processed: 20 out of 96
07/21/2023 02:19:03 [ruzicka:INFO] # test documents processed: 30 out of 96
07/21/2023 02:19:03 [ruzicka:INFO] # test documents processed: 40 out of 96
07/21/2023 02:19:04 [ruzicka:INFO] # test documents processed: 50 out of 96
07/21/2023 02:19:04 [ruzicka:INFO] # test documents processed: 60 out of 96
07/21/2023 02:19:04 [ruzicka:INFO] # test documents processed: 70 out of 96
07/21/2023 02:19:04 [ruzicka:INFO] # test documents processed: 80 out of 96
07/21/2023 02:19:04 [ruzicka:INFO] # test documents processed: 90 out of 96


Accuracy:  0.8854166666666666
AUC:  0.9739583333333334
c@1:  0.9276258680555555
AUC x c@1:  0.9034689444082754


While our final test results are a bit lower, the verifier seems to scale reasonably well to the unseen verification problems in the test set.

# First Order Verification

It is interesting now to compare the GI approach to a first-order verification system, which often yields very competitive results too. Our implementation closely resembles the system proposed by Potha and Stamatatos in 2014 (A Profile-based Method for Authorship Verification). We import and fit this O1 verifier:

In [19]:
from ruzicka.Order1Verifier import Order1Verifier

dev_verifier = Order1Verifier(metric="minmax", base="profile")
dev_verifier.fit(dev_train_X, dev_train_y)
dev_test_scores = dev_verifier.predict_proba(test_X=dev_test_X, test_y=dev_test_y)
print(dev_test_scores)

[7.01701389e-01 7.27071174e-01 3.76331733e-01 2.88794318e-01
 4.90409427e-01 4.27132694e-01 3.89021319e-01 8.98192784e-01
 8.61084515e-01 6.34507071e-01 5.58205461e-01 5.11698323e-01
 7.80041118e-01 7.29013056e-01 2.33797115e-01 6.14428227e-01
 3.00901073e-01 1.54614571e-01 3.77635597e-01 5.08868067e-01
 3.61916662e-01 2.05110484e-01 5.61369766e-01 4.35112027e-01
 3.47092624e-01 4.76222136e-01 2.90966704e-01 4.25212460e-01
 3.30160565e-01 2.82626913e-01 2.93608704e-01 2.50841079e-01
 4.04443303e-01 2.20956493e-01 2.75386099e-01 4.93162012e-01
 1.84686492e-01 4.19353839e-01 1.80123950e-01 3.01737629e-01
 4.68989173e-01 6.70957431e-01 3.42831850e-01 6.01500209e-01
 8.86454495e-01 1.07882863e-01 4.63055368e-01 3.90022158e-01
 2.62076781e-01 2.19900711e-01 3.54165384e-01 3.32777758e-01
 3.84111272e-01 2.31650037e-01 6.09673932e-01 1.75322827e-01
 4.45181997e-01 6.94471446e-01 3.78935565e-01 6.27183739e-01
 5.08018571e-01 6.30246320e-01 8.07823689e-01 5.81900865e-01
 5.38796443e-01 7.263271

Note that in this case, the 'probabilities' returned are only distance-based pseudo-probabilities and don't lie in the range of 0-1. Applying the score shifter is therefore quintessential with O1, since it will scale the distances to a more useful range:

In [20]:
shifter = ScoreShifter()
shifter.fit(predicted_scores=dev_test_scores, ground_truth_scores=dev_gt_scores)
dev_test_scores = shifter.transform(dev_test_scores)
print(dev_test_scores)

07/21/2023 02:19:08 [ruzicka:INFO] p1 for optimal combo: 0.4
07/21/2023 02:19:08 [ruzicka:INFO] p2 for optimal combo: 0.47
07/21/2023 02:19:08 [ruzicka:INFO] AUC for optimal combo: 0.904513888888889
07/21/2023 02:19:08 [ruzicka:INFO] c@1 for optimal combo: 0.8741319444444444


[0.8419017359935173, 0.8553477220231838, 0.15053269329768626, 0.11551772735208198, 0.7299169963137953, 0.5, 0.15560852751315557, 0.9460421752963407, 0.9263747928625592, 0.8062887475417948, 0.765848894174316, 0.7412001114343534, 0.8834217927147184, 0.85637691987103, 0.09351884596681884, 0.7956469605047912, 0.12036042901403372, 0.061845828485266735, 0.15105423890906988, 0.7397000756258421, 0.144766664715685, 0.08204419361515838, 0.7675259759154384, 0.5, 0.13883704976265296, 0.7223977320117336, 0.11638668160158078, 0.5, 0.13206422617873098, 0.1130507651392888, 0.1174434814338157, 0.10033643156983099, 0.5, 0.08838259739667048, 0.11015443956248164, 0.7313758661859044, 0.07387459694578445, 0.5, 0.07204958015563766, 0.12069505173174132, 0.5, 0.8256074382194236, 0.13713273983072913, 0.7887951107272342, 0.9398208822606706, 0.04315314523239408, 0.5, 0.15600886329938662, 0.10483071236672521, 0.08796028455792115, 0.14166615370159674, 0.1331111032379135, 0.15364450871256846, 0.09266001468198548, 0.

And again, we are now ready to test the performance of O1 on the test problems.

In [21]:
train_data, test_data = load_pan_dataset(D + "test")
train_labels, train_documents = zip(*train_data)
test_labels, test_documents = zip(*test_data)

# vectorize:
vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)
train_X = vectorizer.fit_transform(train_documents)
test_X = vectorizer.transform(test_documents)

# encode author labels:
label_encoder = LabelEncoder()
label_encoder.fit(train_labels + test_labels)
train_y = np.array(label_encoder.transform(train_labels), dtype="int")
test_y = np.array(label_encoder.transform(test_labels), dtype="int")

# fit and predict a verifier on the test data:
test_verifier = Order1Verifier(metric="minmax", base="profile")
test_verifier.fit(train_X, train_y)
test_scores = test_verifier.predict_proba(test_X=test_X, test_y=test_y)

# load the ground truth:
test_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "test", "truth.txt")), labels=test_labels
)

# apply the optimzed score shifter:
test_scores = shifter.transform(test_scores)

test_acc_score, test_auc_score, test_c_at_1_score = pan_metrics(
    prediction_scores=test_scores, ground_truth_scores=test_gt_scores
)

print("Accuracy: ", test_acc_score)
print("AUC: ", test_auc_score)
print("c@1: ", test_c_at_1_score)
print("AUC x c@1: ", test_auc_score * test_c_at_1_score)

Accuracy:  0.8125
AUC:  0.8899739583333334
c@1:  0.830078125
AUC x c@1:  0.7387479146321615
